In [1]:
import json, gzip,ujson
import pandas as pd 
import numpy as np, toml
from typing import Tuple, List, Dict
import mysql.connector as mariadb

In [23]:
selected_labdoc = 445494

summary_all  = pd.read_csv('/Users/anis/test_labnbook/semantic_indicator/data/tmp/reports/summary_all.csv', index_col = [0])
summary_labdoc = summary_all[summary_all["id_labdoc"] == int(selected_labdoc)]
summary_labdoc['action_time'] = pd.to_datetime(summary_labdoc['action_time'])
summary_labdoc['edition_time'] = summary_labdoc['action_time'].diff()
# Convert to integer value:
summary_labdoc["edition_time"] = summary_labdoc["edition_time"]
summary_labdoc["edition_time"] 

/var/folders/qz/t1vzp79913l2sfbf0b3d9sdr0000gn/T/ipykernel_13220/483277202.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  summary_labdoc['action_time'] = pd.to_datetime(summary_labdoc['action_time'])
/var/folders/qz/t1vzp79913l2sfbf0b3d9sdr0000gn/T/ipykernel_13220/483277202.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  summary_labdoc['edition_time'] = summary_labdoc['action_time'].diff()
/var/folders/qz/t1vzp79913l2sfbf0b3d9sdr0000gn/T/ipykernel_13220/483277202.py:8: SettingWithCopyWarning: 
A va

7089               NaT
7090               NaT
7091   0 days 01:12:42
7092   0 days 00:03:02
7093   0 days 00:01:29
7094   1 days 21:26:58
Name: edition_time, dtype: timedelta64[ns]

In [22]:
summary_labdoc['edition_time'] =pd.to_timedelta(summary_labdoc['edition_time'])

7089               NaT
7090               NaT
7091   0 days 01:12:42
7092   0 days 00:03:02
7093   0 days 00:01:29
7094   1 days 21:26:58
Name: edition_time, dtype: timedelta64[ns]

In [25]:
summary_labdoc['edition_time'] 

7089            NaN
7090            NaN
7091      72.700000
7092       3.033333
7093       1.483333
7094    2726.966667
Name: edition_time, dtype: float64

In [26]:
summary_labdoc[['eqc','coec']]

,eqc,coec
7089,0.000000,0.000000
7090,0.000000,0.000000
7091,0.042793,0.045333
7092,0.066413,0.070133
7093,0.066413,0.070133
7094,0.118471,0.150075


In [28]:
summary_labdoc.drop(['id_mission','id_report'],axis =1, inplace = True)


/var/folders/qz/t1vzp79913l2sfbf0b3d9sdr0000gn/T/ipykernel_13220/3131653315.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  summary_labdoc.drop(['id_mission','id_report'],axis =1, inplace = True)


In [29]:
summary_labdoc

,id_labdoc,id_trace,action_time,n_users,teacher,n_tokens,n_segments,eqc,coec,user,sim,edition_time
7089,445494,0,NaT,1,1,86,1,0.000000,0.000000,ens,1.00,NaN
7090,445494,7481881,2021-03-16 10:05:29,2,1,112,3,0.000000,0.000000,12673,0.69,NaN
7091,445494,7482763,2021-03-16 11:18:11,3,1,111,3,0.042793,0.045333,12661,1.00,72.700000
7092,445494,7482817,2021-03-16 11:21:13,3,1,121,6,0.066413,0.070133,12673,1.00,3.033333
7093,445494,7482845,2021-03-16 11:22:42,3,1,121,6,0.066413,0.070133,12661,1.00,1.483333
7094,445494,7505160,2021-03-18 08:49:40,3,1,150,8,0.118471,0.150075,12673,0.98,2726.966667


In [35]:
selected_mission =451


array([35531, 35532, 35533, 35534, 35535, 35536, 35549, 35550, 35551,
       35552, 35553, 35554, 35567, 35568, 35569, 35570, 35571, 35572,
       35585, 35586, 35587, 35588, 35589, 35590, 35693, 35694, 35695,
       35696, 35697, 35698, 35797, 35798, 35799, 35800, 35801, 35802,
       35815, 35816, 35817, 35818, 35819, 35820, 35917, 35918, 35919,
       35920, 35921, 35922, 35936, 35937, 35938, 35939, 35940, 35941,
       36453, 36454, 36455, 36456, 36457, 36458])

In [51]:
s = pd.Series(['anis@hddgmail.com', 'b2', 'c3'])
s.str.extract(r'(/^\S+@\S+$/)')


,0
0,NaN
1,NaN
2,NaN


In [52]:
s

0    anis@hddgmail.com
1                   b2
2                   c3
dtype: object